# HEMOC Recovery Phase D.3–D.6: GPU Experiments

**One-click notebook.** Runtime > Run All. Come back in ~40 min.

| Experiment | What it tests | Expected |
|-----------|--------------|----------|
| D.3 Simple Grating | Encoder sophistication contribution | 0.5–0.7 corr |
| D.4 Hypercube Ablation | Per-angle path routing (3 variants) | Hybrid > Direct > All-hyper |
| D.5 Golden Ratio | Is phi mathematically necessary? | **Unknown** |
| D.6 Resolution | Optimal pixel density (32/64/128) | Diminishing returns above 64 |

Results auto-save to `results/` and get pushed back to the branch.

## 0. Setup

In [1]:
# Verify GPU is available
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
    print(f"VRAM: {vram / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Go to Runtime > Change runtime type > T4 GPU")
    print("Experiments will still run on CPU but will be slower.")

PyTorch: 2.9.0+cu128
CUDA available: True
GPU: Tesla T4
VRAM: 15.6 GB


In [2]:
# Clone repo and checkout branch
import os
if not os.path.exists('/content/HEMOC-Stain-Glass-Flower'):
    !git clone https://github.com/Domusgpt/HEMOC-Stain-Glass-Flower.git /content/HEMOC-Stain-Glass-Flower
else:
    print("Repo already cloned")

os.chdir('/content/HEMOC-Stain-Glass-Flower')
!git checkout claude/recovery-plan-phases-F14wo
!git pull origin claude/recovery-plan-phases-F14wo
!pip install -q scipy matplotlib
print("\nSetup complete.")

Cloning into '/content/HEMOC-Stain-Glass-Flower'...
remote: Enumerating objects: 7013, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 7013 (delta 261), reused 233 (delta 225), pack-reused 6718 (from 3)
Receiving objects: 100% (7013/7013), 77.78 MiB | 25.73 MiB/s, done.
Resolving deltas: 100% (1764/1764), done.
Branch 'claude/recovery-plan-phases-F14wo' set up to track remote branch 'claude/recovery-plan-phases-F14wo' from 'origin'.
Switched to a new branch 'claude/recovery-plan-phases-F14wo'
From https://github.com/Domusgpt/HEMOC-Stain-Glass-Flower
 * branch            claude/recovery-plan-phases-F14wo -> FETCH_HEAD
Already up to date.

Setup complete.


## 1. D.3 — Simple Grating Baseline
Simple sinusoidal gratings (no hypercube, no golden ratio). Isolates how much HybridEncoder sophistication helps.

In [3]:
os.chdir('/content/HEMOC-Stain-Glass-Flower')
!python demos/baseline_simple_encoder.py

D.3: SIMPLE VISUAL BASELINE (Grating-Only Encoder)
Timestamp: 2026-02-18 00:51:10
Device: cuda
Purpose: Isolate contribution of HybridEncoder sophistication.
         Simple gratings = sinusoidal only, no hypercube/golden ratio.

Generating 10000 simple-grating samples...

Training CNN on SIMPLE grating patterns
  Model: 430,918 parameters
  [SIMPLE] Epoch  10: loss=0.69850, corr=0.8619, pass=6/6
  [SIMPLE] Epoch  20: loss=0.42467, corr=0.8903, pass=6/6
  [SIMPLE] Epoch  30: loss=0.28372, corr=0.8916, pass=6/6
  [SIMPLE] Epoch  40: loss=0.20272, corr=0.8895, pass=6/6
  [SIMPLE] Epoch  50: loss=0.17778, corr=0.8917, pass=6/6

  Simple encoder results:
  Angle    Corr       MAE        Pass
  0(lin)   0.7490     0.8603     PASS
  1(lin)   0.9829     0.1933     PASS
  2(lin)   0.9842     0.1872     PASS
  3(hyp)   0.7764     0.6465     PASS
  4(hyp)   0.9480     0.3570     PASS
  5(hyp)   0.9096     0.4267     PASS

  Average: 0.8917, Passing: 6/6

Saved to: results/baseline_simple_encoder

In [4]:
# Quick check D.3 results
import json
with open('results/baseline_simple_encoder_results.json') as f:
    d3 = json.load(f)
print(f"D.3 Simple Grating: avg_corr={d3['results']['avg_corr']:.4f}, passing={d3['results']['passing']}/6")
print(f"  Device: {d3['device']}")

D.3 Simple Grating: avg_corr=0.8917, passing=6/6
  Device: cuda


## 2. D.4 — Hypercube Path Ablation
Three variants: all-direct, hybrid (original), all-hypercube. Quantifies per-angle path contribution.

In [5]:
os.chdir('/content/HEMOC-Stain-Glass-Flower')
!python demos/ablation_hypercube.py

D.4: HYPERCUBE PATH ABLATION
Timestamp: 2026-02-18 00:53:28
Device: cuda
Purpose: Compare all-direct vs hybrid vs all-hypercube routing.

Variant: ALL_DIRECT
  Generating 10000 samples...
  [ALL_DIRECT] Epoch  10: loss=0.60697, corr=0.9052, pass=6/6
  [ALL_DIRECT] Epoch  20: loss=0.41387, corr=0.9311, pass=6/6
  [ALL_DIRECT] Epoch  30: loss=0.23877, corr=0.9248, pass=6/6
  [ALL_DIRECT] Epoch  40: loss=0.15191, corr=0.9464, pass=6/6
  [ALL_DIRECT] Epoch  50: loss=0.13503, corr=0.9503, pass=6/6

  all_direct results:
  Angle    Corr       MAE        Pass
  0(lin)   0.8100     0.6952     PASS
  1(lin)   0.9516     0.3322     PASS
  2(lin)   0.9523     0.2969     PASS
  3(hyp)   0.9952     0.1338     PASS
  4(hyp)   0.9964     0.1202     PASS
  5(hyp)   0.9964     0.1250     PASS

  Average: 0.9503, Passing: 6/6

Variant: HYBRID_ORIGINAL
  Generating 10000 samples...
  [HYBRID_ORIGINAL] Epoch  10: loss=0.93634, corr=0.7303, pass=5/6
  [HYBRID_ORIGINAL] Epoch  20: loss=0.57925, corr=0.7885,

In [6]:
# Quick check D.4 results
with open('results/ablation_hypercube_results.json') as f:
    d4 = json.load(f)
print("D.4 Hypercube Ablation:")
for name, v in d4['variants'].items():
    print(f"  {name:<20} avg_corr={v['avg_corr']:.4f}, passing={v['passing']}/6")

D.4 Hypercube Ablation:
  all_direct           avg_corr=0.9503, passing=6/6
  hybrid_original      avg_corr=0.8810, passing=6/6
  all_hypercube        avg_corr=0.4962, passing=2/6


## 3. D.5 — Golden Ratio Substitution Ablation
Replace phi with sqrt(2), e, pi, rationals. Is the golden ratio mathematically necessary or just convenient?

In [7]:
os.chdir('/content/HEMOC-Stain-Glass-Flower')
!python demos/ablation_golden_ratio.py

D.5: GOLDEN RATIO SUBSTITUTION ABLATION
Timestamp: 2026-02-18 01:01:11
Device: cuda
Purpose: Is phi mathematically necessary or just a convenient irrational?

Variant: GOLDEN_ORIGINAL
  Constants: phi=1.618034, 1/phi=0.618034
  Generating 10000 samples...
  [GOLDEN_ORIGINAL] Epoch  10: loss=0.94606, corr=0.8148, pass=6/6
  [GOLDEN_ORIGINAL] Epoch  20: loss=0.59770, corr=0.7752, pass=6/6
  [GOLDEN_ORIGINAL] Epoch  30: loss=0.31963, corr=0.8553, pass=6/6
  [GOLDEN_ORIGINAL] Epoch  40: loss=0.19666, corr=0.8731, pass=6/6
  [GOLDEN_ORIGINAL] Epoch  50: loss=0.16444, corr=0.8773, pass=6/6

  golden_original results:
  Angle    Corr       MAE        Pass
  0(lin)   0.7751     0.8000     PASS
  1(lin)   0.8936     0.5273     PASS
  2(lin)   0.9147     0.4376     PASS
  3(hyp)   0.8899     0.3119     PASS
  4(hyp)   0.9003     0.3699     PASS
  5(hyp)   0.8902     0.3576     PASS

  Average: 0.8773, Passing: 6/6

Variant: GOLDEN_SQRT2
  Constants: sqrt(2)=1.414214, 1/sqrt(2)=0.707107
  Generat

In [8]:
# Quick check D.5 results
with open('results/ablation_golden_ratio_results.json') as f:
    d5 = json.load(f)
print("D.5 Golden Ratio Ablation:")
for name, v in d5['variants'].items():
    print(f"  {name:<20} avg_corr={v['avg_corr']:.4f}, passing={v['passing']}/6")
print(f"\nVerdict: {d5['verdict']}")

D.5 Golden Ratio Ablation:
  golden_original      avg_corr=0.8773, passing=6/6
  golden_sqrt2         avg_corr=0.8803, passing=6/6
  golden_e             avg_corr=0.8642, passing=6/6
  golden_pi            avg_corr=0.8801, passing=6/6
  golden_rational      avg_corr=0.8723, passing=6/6

Verdict: PHI IS NOT SPECIAL: Other constants perform equally well


## 4. D.6 — Resolution Sweep Ablation
Test at 32x32, 64x64, 128x128. Find optimal resolution for information density.

In [ ]:
os.chdir('/content/HEMOC-Stain-Glass-Flower')
!python demos/ablation_resolution.py

D.6: RESOLUTION SWEEP ABLATION
Timestamp: 2026-02-18 01:14:04
Device: cuda
Purpose: Find optimal resolution for information density.
Resolutions: [32, 64, 128]

Resolution: 32x32 (3,072 values per pattern)
  Generating 7500 samples at 32x32...
  Generated in 34.0s
  Pattern shape: (6000, 3, 32, 32)
  Model: 430,918 parameters
  [RES-32] Epoch  10: loss=1.08385, corr=0.5801, pass=5/6
  [RES-32] Epoch  20: loss=0.49003, corr=0.7397, pass=6/6


In [ ]:
# Quick check D.6 results
with open('results/ablation_resolution_results.json') as f:
    d6 = json.load(f)
print("D.6 Resolution Ablation:")
for name, v in d6['variants'].items():
    print(f"  {name:<14} avg_corr={v['avg_corr']:.4f}, passing={v['passing']}/6")
print(f"\nVerdict: {d6['verdict']}")

## 5. Summary Dashboard

In [ ]:
import json
import os

os.chdir('/content/HEMOC-Stain-Glass-Flower')

print("=" * 70)
print("HEMOC RECOVERY PHASE D: COMPLETE RESULTS SUMMARY")
print("=" * 70)

# D.3
with open('results/baseline_simple_encoder_results.json') as f:
    d3 = json.load(f)
print(f"\nD.3 Simple Grating Baseline:")
print(f"  Avg correlation: {d3['results']['avg_corr']:.4f}")
print(f"  Passing angles:  {d3['results']['passing']}/6")
print(f"  Device: {d3['device']}, Time: {d3['config'].get('total_time_s', '?'):.0f}s")

# D.4
with open('results/ablation_hypercube_results.json') as f:
    d4 = json.load(f)
print(f"\nD.4 Hypercube Path Ablation:")
best_hyp = max(d4['variants'].items(), key=lambda x: x[1]['avg_corr'])
for name, v in d4['variants'].items():
    marker = " << BEST" if name == best_hyp[0] else ""
    print(f"  {name:<20} corr={v['avg_corr']:.4f}  pass={v['passing']}/6{marker}")

# D.5
with open('results/ablation_golden_ratio_results.json') as f:
    d5 = json.load(f)
print(f"\nD.5 Golden Ratio Ablation:")
best_gr = max(d5['variants'].items(), key=lambda x: x[1]['avg_corr'])
for name, v in d5['variants'].items():
    marker = " << BEST" if name == best_gr[0] else ""
    print(f"  {name:<20} corr={v['avg_corr']:.4f}  pass={v['passing']}/6{marker}")
print(f"  Verdict: {d5['verdict']}")

# D.6
with open('results/ablation_resolution_results.json') as f:
    d6 = json.load(f)
print(f"\nD.6 Resolution Ablation:")
best_res = max(d6['variants'].items(), key=lambda x: x[1]['avg_corr'])
for name, v in d6['variants'].items():
    marker = " << BEST" if name == best_res[0] else ""
    print(f"  {v['resolution']:>3}x{v['resolution']:<3}  corr={v['avg_corr']:.4f}  pass={v['passing']}/6{marker}")
print(f"  Verdict: {d6['verdict']}")

# Context: prior results
print(f"\n{'=' * 70}")
print("CONTEXT: Previously completed results")
print(f"{'=' * 70}")
print(f"  HybridEncoder + CNN (music):        0.916 avg corr, 6/6 passing")
print(f"  Domain-diverse (cross-domain):       0.729 avg corr")
print(f"  D.1 MLP ceiling (no visual):         ~0.64 avg corr, 5/6 passing")
print(f"  D.2 Random pattern floor:            -0.001 avg corr, 0/6 passing")
print(f"\nAll Recovery Phase D experiments complete.")

## 6. Commit & Push Results

In [ ]:
os.chdir('/content/HEMOC-Stain-Glass-Flower')

# Verify all result files exist
expected = [
    'results/baseline_simple_encoder_results.json',
    'results/ablation_hypercube_results.json',
    'results/ablation_golden_ratio_results.json',
    'results/ablation_resolution_results.json',
]
all_exist = True
for f in expected:
    exists = os.path.exists(f)
    status = 'OK' if exists else 'MISSING'
    print(f"  {status}: {f}")
    if not exists:
        all_exist = False

if all_exist:
    print("\nAll result files present. Committing...")
    !git add results/baseline_simple_encoder_results.json
    !git add results/ablation_hypercube_results.json
    !git add results/ablation_golden_ratio_results.json
    !git add results/ablation_resolution_results.json
    !git commit -m "Add Recovery Phase D.3-D.6 GPU experiment results"
    !git push origin claude/recovery-plan-phases-F14wo
    print("\nDone! Results pushed to branch.")
else:
    print("\nSome results missing. Fix the failed experiments above before pushing.")